In [ ]:
!pip3 install "faust"

In [ ]:
!pip3 install 'git+https://github.com/logicalclocks/hopsworks-api@main#egg=hopsworks&subdirectory=python'

In [7]:
import faust
import hopsworks
import ssl


In [17]:
help(faust)

Help on package faust:

NAME
    faust - Python Stream processing.

PACKAGE CONTENTS
    __main__
    _cython (package)
    agents (package)
    app (package)
    assignor (package)
    auth
    channels
    cli (package)
    events
    exceptions
    fixups (package)
    joins
    livecheck (package)
    models (package)
    sensors (package)
    serializers (package)
    stores (package)
    streams
    tables (package)
    topics
    transport (package)
    types (package)
    utils (package)
    web (package)
    windows
    worker

CLASSES
    abc.ABC(builtins.object)
        faust.types.models.ModelOptions
        faust.types.settings.Settings
    collections.abc.AsyncIterable(builtins.object)
        faust.types.streams.StreamT(collections.abc.AsyncIterable, faust.types.streams.JoinableT, mode.types.services.ServiceT)
            faust.streams.Stream(faust.types.streams.StreamT, mode.services.Service)
    collections.abc.AsyncIterator(collections.abc.AsyncIterable)
        faust

In [10]:
import hsfs

HOST="34.78.148.2"
PROJECT="demo_fs_meb10000"

connection = hsfs.connection(
    host=HOST,
    project=PROJECT,
    api_key_value="9tybM5t8Vj5ZPH6L.badVzfTTdelhCTZLDHthlHL7ITfDTI9IRsplEQ91ViHmBdn5JFDXJsJ47s7UtklL",
    engine="python"
)
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [11]:
BASE_PATH = "/tmp/" + HOST + "/" + PROJECT + "/"
CA_FILE = BASE_PATH + "ca_chain.pem"
CERT_FILE = BASE_PATH + "client_cert.pem"
KEY_FILE = BASE_PATH + "client_key.pem"
ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH, cafile=CA_FILE)
ssl_context.load_cert_chain(CERT_FILE, keyfile=KEY_FILE)
# ssl_context = kafka.get_ssl_context()


In [12]:
# raw -> avro??
BROKER_URL = "kafka://" + HOST + ":9092"
app = faust.App('empty', broker=BROKER_URL, #store='rocksdb://', 
  value_serializer='json', broker_credentials=ssl_context)    #A


In [14]:
cc_topic = app.topic('empty', key_type=str, value_type=str)


In [15]:
# This sends events every 1 second to the process method above.
@app.timer(interval=10.0)
async def topic_writer(app):
    await cc_topic.send(
        value="hello",
    )


In [ ]:
@app.agent(ingest)
async def test(stream):
    async for values in stream.take(100, within=10):
        yield values

In [ ]:
cc_topic = app.topic('cc_transactions', key_type=str, value_type=cc_transaction)

# Order is a json serialized dictionary,
# having these fields:

class cc_transaction(faust.Record, serializer=’avro’):
    cc_num: str
    category: str
    amount: float
    transaction_ts: str

count_by_cc_num = app.Table('cc_count', default=int)

@app.agent(cc_topic)
async def process(cc_trans: faust.Stream[cc_transaction]) -> None:
    async for trans in cc_trans.group_by(cc_transaction.cc_num):
        count_by_cc_num[trans.cc_num] += 1


# This sends events every 1 second to the process method above.
@app.timer(interval=1.0)
async def topic_writer(app):
    await cc_topic.send(
        value=cc_transaction(cc_num='Faust', category='you', amount=’’, transaction_ts=’’),
    )


# Have this one only write in batchs to HSFS
@app.agent(orders_topic)
async def process_order(orders):
    async for order in orders:
        # process each order using regular Python
        total_price = order.price * order.quantity
        await send_order_received_email(order.account_id, order)



In [16]:
app.main()

Usage: ipykernel_launcher.py [OPTIONS] COMMAND [ARGS]...
Try 'ipykernel_launcher.py --help' for help.

Error: Missing command.


AssertionError: 